# Credit
Fork from https://www.kaggle.com/code/rsuhara/ai-generated-text-detection-quick-baseline

Inspired by VLADIMIR DEMIDOV's work : <br>
https://www.kaggle.com/code/yekenot/llm-detect-by-regression

For the training data we shall use the "RDizzl3 seven" dataset (v1) which can be found in the "LLM: 7 prompt training dataset" https://www.kaggle.com/datasets/carlmcbrideellis/llm-7-prompt-training-dataset



# Importing library

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Importing files and Feature Engineering

In [ ]:
# external_df = pd.read_csv("/kaggle/input/daigt-external-dataset/daigt_external_dataset.csv", sep=',')
# print(external_df.shape)
# external_df = external_df.rename(columns={'generated': 'label'})
# external_df = external_df[["source_text"]]
# external_df.columns = ["text"]
# external_df['text'] = external_df['text'].str.replace('\n', '')
# external_df["label"] = 1

In [ ]:
train1 = pd.read_csv("/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv")
train2 = pd.read_csv("/kaggle/input/daigt-proper-train-dataset/train_drcat_02.csv")
train3 = pd.read_csv("/kaggle/input/daigt-proper-train-dataset/train_drcat_03.csv")

train=pd.concat([train1, train2, train3])
test = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv')

df = pd.concat([train['text'], test['text']], axis=0)

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df)

# Logistic Regression

In [ ]:
lr_model = LogisticRegression(solver="liblinear")

# SGD

In [ ]:
sgd_model = SGDClassifier(max_iter=1000, tol=1e-3, loss="modified_huber")

# NB

In [ ]:
nb_model = MultinomialNB()

# SVC

In [ ]:
# svc_model = SVC()

# Voting Classifier

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

# Assuming you have already defined and trained your individual models (lr_model, sgd_model, nb_model, svc_model)
# and loaded your data into X and train.label

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X[:train.shape[0]], train.label, test_size=0.2, random_state=42)

# Create the ensemble model
ensemble = VotingClassifier(estimators=[('lr', lr_model), ('sgd', sgd_model), ('nb', nb_model)], voting='soft')
ensemble.fit(X_train, y_train)

# Predictions
y_pred = ensemble.predict(X_test)

# Create confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print precision, recall, and F1 score
print(classification_report(y_test, y_pred))

# Plot confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
X_test = vectorizer.transform(test['text'])

y_test = test['label']

# Predictions
y_pred = ensemble.predict(X_test)

# Create confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print precision, recall, and F1 score
print(classification_report(y_test, y_pred))

# Plot confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# pd.DataFrame({'id':test["id"],'generated':preds_test}).to_csv('submission.csv', index=False)